In [1]:
import pyreadr
import rpy2
import pyarrow.parquet as pq
import os, os.path
import numpy as np
import pandas as pd
import json
import time
from collections import defaultdict
import pyreadr
import rpy2
import pyarrow.parquet as pq
import re
import matplotlib.pyplot as plt
import os

In [2]:
os.environ['R_HOME'] = 'D:/Users/Chenhao/anaconda3/envs/RPSHA/Lib/R'
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects

In [3]:
RPSHA = importr("RPSHA")

In [4]:
####Read sites information
sites = pd.read_csv('D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/sites_dlt.csv')
sites_id = sites['ID'].values

decimals = 2    
sites['X'] = sites['X'].apply(lambda x: round(x, decimals))
sites['Y'] = sites['Y'].apply(lambda x: round(x, decimals))

sites_x = sites['X'].values
sites_y = sites['Y'].values

print(sites)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/sites_dlt.csv'

In [5]:
####Input variables
one_pt_in_region = [38.05, -121.54]  # The 'central coordiates' around a map around which the max_distance is determined
max_distance = 200 # The max. dist outside of which the rupture around the 'one_pt_in_region' will not be considered
im_name = 'PGA'

rp_name = ['10rp', '50rp', '100rp', '225rp', '475rp', '2475rp']
rp_value = [10, 50, 100, 225, 475, 2475]
max_d = 150

fault_threshold = 4
point_threshold = 2

example_site_index = [0,56,154] ####Sites indices you wanna compare harzard matching curve

column_index = 20

nshmp_hazard_loc = 'D:/Users/JupyterNotebook/StoEvtHaz/RegionalHazard_LASSO/deagg/'

z1 = [-999]*len(sites)
vs = [350]*len(sites) # change here, the VS30 info.

In [6]:
####Read all events in UCERF3
result = pyreadr.read_r('D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/FM31_Pts.rda')
FM31_Pts = result["FM31_Pts"]


result = pyreadr.read_r('D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/FM31_Secs.rda')
FM31_Secs = result["FM31_Secs"]

result = pyreadr.read_r('D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/FM32_Pts.rda')
FM32_Pts = result["FM32_Pts"]


result = pyreadr.read_r('D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/FM32_Secs.rda')
FM32_Secs = result["FM32_Secs"]

#FM31 Ruptures
FM31_Rups = pq.read_table('D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/FM31_Rups.parquet').to_pandas()
FM31_Rups

#FM32 Ruptures
FM32_Rups = pq.read_table('D:/Users/JupyterNotebook/RegionalHazard_LASSO/input/FM32_Rups.parquet').to_pandas()
#FM32_Rups

FM31_Rups

,Index,Indices,Mag,Rate,Type,Depth,Dip,Rake,Width,ID
0,[0:1],"[0, 1]",[6.449],[3.3631184e-05],[SINGLE],[1.3],[50.0],[-90.0],[15.273],[1]
1,[0:2],"[0, 1, 2]",[6.638],[1.534016e-05],[SINGLE],[1.3],[50.0],[-90.0],[15.273],[2]
2,[0:3],"[0, 1, 2, 3]",[6.78],[1.090303e-05],[SINGLE],[1.3],[50.0],[-90.0],[15.273],[3]
3,[0:4],"[0, 1, 2, 3, 4]",[6.893],[7.3397665e-06],[SINGLE],[1.3],[50.0],[-90.0],[15.273],[4]
4,[0:5],"[0, 1, 2, 3, 4, 5]",[6.988],[3.9774597e-05],[SINGLE],[1.3],[50.0],[-90.0],[15.273],[5]
...,...,...,...,...,...,...,...,...,...,...
253604,[2473:2475],"[2473, 2474, 2475]",[6.496],[4.4253078e-07],[SINGLE],[1.2],[60.0],[-90.0],[12.471],[253605]
253605,[2473:2476],"[2473, 2474, 2475, 2476]",[6.637],[7.0331052e-06],[SINGLE],[1.2],[60.0],[-90.0],[12.471],[253606]
253606,[2474:2475],"[2474, 2475]",[6.303],[6.8217923e-07],[SINGLE],[1.2],[60.0],[-90.0],[12.471],[253607]
253607,[2474:2476],"[2474, 2475, 2476]",[6.496],[1.5162712e-05],[SINGLE],[1.2],[60.0],[-90.0],[12.471],[253608]


In [7]:
####Extract staring and ending trace lon and lat from FM31 and FM32 Secs

pattern = re.compile(r'-?\d+\.\d+,-?\d+\.\d+(?=,)')
FM31_BeEn_Trace = []
FM32_BeEn_Trace = []

for i in range(len(FM31_Secs)):
    # Find the first and last matches of the pattern in the input string
    matches = pattern.findall(FM31_Secs.iloc[i][7])

    # Extract the first and last lon and lat combinations and convert to float
    first_lon, first_lat = map(float, matches[0].split(','))
    last_lon, last_lat = map(float, matches[-1].split(','))
    

    # Create a list with the first and last lon and lat combinations
    lon_lat_list = [(first_lon, first_lat), (last_lon, last_lat)]
    
    FM31_BeEn_Trace.append(lon_lat_list)

for i in range(len(FM32_Secs)):
    # Find the first and last matches of the pattern in the input string
    matches = pattern.findall(FM32_Secs.iloc[i][7])

    # Extract the first and last lon and lat combinations and convert to float
    first_lon, first_lat = map(float, matches[0].split(','))
    last_lon, last_lat = map(float, matches[-1].split(','))
    

    # Create a list with the first and last lon and lat combinations
    lon_lat_list = [(first_lon, first_lat), (last_lon, last_lat)]
    
    FM32_BeEn_Trace.append(lon_lat_list)
    
print(len(FM31_BeEn_Trace), len(FM32_BeEn_Trace))

2606 2664


In [82]:
####IMPORT FUNCTIONS IN R PACKAGES
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()

revents_hazmat_calc = robjects.r['events_hazmat_calc']
rscenario_selection = robjects.r['scenario_selection']
rflts_filter = robjects.r['flts_filter']
rpts_filter = robjects.r['pts_filter']

flt_fm31f = rflts_filter(one_pt_in_region[0], one_pt_in_region[1], max_dist = max_distance, branch = 1) ##Any one point in your region; maximum distance
flt_fm31f = ro.conversion.get_conversion().rpy2py(flt_fm31f)


flt_fm32f = rflts_filter(one_pt_in_region[0], one_pt_in_region[1],max_dist = max_distance, branch = 2)
flt_fm32f = ro.conversion.rpy2py(flt_fm32f)
flt_fm32f

flt_fm32p = rpts_filter(one_pt_in_region[0], one_pt_in_region[1],max_dist = max_distance, branch = 2)
flt_fm32p = ro.conversion.rpy2py(flt_fm32p)
flt_fm32p

flt_fm31p = rpts_filter(one_pt_in_region[0], one_pt_in_region[1],max_dist = max_distance, branch = 1)
flt_fm31p = ro.conversion.rpy2py(flt_fm31p)


In [78]:
flt_fm31f

,Index,Mag,Rate,Depth,Dip,Rake,Width,Rrup,Rjb,ID
1,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.676,9.614238e-09,0.850,80.8,165.1,13.865,195.407038,195.407038,5827
2,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.677,1.253341e-08,0.874,80.8,165.1,13.834,190.118244,190.118244,5828
3,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.678,6.824738e-09,0.898,80.8,165.2,13.803,184.861774,184.861774,5829
4,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.679,1.230271e-08,0.925,80.8,165.2,13.768,179.640498,179.640498,5830
5,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.680,7.191361e-09,0.959,80.9,165.3,13.724,174.491278,174.491278,5831
...,...,...,...,...,...,...,...,...,...,...
178394,2473:2475,6.496,4.425308e-07,1.200,60.0,-90.0,12.471,138.823718,138.823718,253605
178395,2473:2476,6.637,7.033105e-06,1.200,60.0,-90.0,12.471,138.823718,138.823718,253606
178396,2474:2475,6.303,6.821792e-07,1.200,60.0,-90.0,12.471,144.939447,144.939447,253607
178397,2474:2476,6.496,1.516271e-05,1.200,60.0,-90.0,12.471,144.939447,144.939447,253608


# PGA matching input files

In [9]:
#####Target Hazard files

###PGA
###Change your PSHA result file path
PGA_HAZ = pd.read_csv(nshmp_hazard_loc + '/10rp/PGA/curves.csv', header = None) # Either return priod gets the same results

IM_level = PGA_HAZ.loc[0].tolist()[3:]
IM1 = [im_name]*len(sites)*len(IM_level)   #### Change to the name of IM your are interested in

###IM level
IM_L = []
for i in range(len(IM_level)):
    for j in range(len(sites)):
        IM_L.append(str(IM_level[i]))
len(IM_L)

#### Site names
site_name_single = PGA_HAZ.iloc[:,0].tolist()[1:]
site_name = site_name_single*len(IM_level)


####Annual exceedance Rate
AER = []
for i in range(3, PGA_HAZ.shape[1]):
    temp = PGA_HAZ.iloc[:,i].tolist()[1:]
    AER = AER + temp
    
Y_Haz = pd.DataFrame(list(zip(IM1, IM_L, site_name, AER)), columns=['IM_type', 'IM_level', 'SiteName', 'AnnualExceedanceRate'])
Y_Haz

####Adjust the fitting range
Y_Haz = Y_Haz.iloc[785:2512,:]    ####Select your range of interest

In [10]:
Y_Haz.to_csv(nshmp_hazard_loc+'/PGA_yhaz.csv')

In [11]:
##### Site Table Input files
SiteTable = pd.DataFrame(list(zip(site_name_single, sites_x, sites_y, vs, z1)), columns=['SiteName', 'Lon', 'Lat', 'Vs30', 'z1'])
SiteTable

,SiteName,Lon,Lat,Vs30,z1
0,Delta Site1,-121.90,38.05,350,-999
1,Delta Site2,-121.90,38.00,350,-999
2,Delta Site3,-121.85,38.09,350,-999
3,Delta Site4,-121.85,38.05,350,-999
4,Delta Site5,-121.85,38.00,350,-999
...,...,...,...,...,...
152,Delta Site153,-121.27,37.78,350,-999
153,Delta Site154,-121.27,37.73,350,-999
154,Delta Site155,-121.27,37.69,350,-999
155,Delta Site156,-121.22,37.73,350,-999


In [12]:
SiteTable.to_csv(nshmp_hazard_loc+'/site_table.csv')

In [13]:
######Deaggregation results input files

SiteName = []
R_distance = []
Magnitude = []
HazardContribution = []
Epsilon_bar = []
IM_type = []
IM_level = []
ReturnPeriod = []
AnnualExceedanceRate = []
    
for j in range(len(rp_name)):
    rp_path_pga = rp_name[j]+'/PGA/deagg'     ######You may wanna modify this to your scenario
    RP = rp_value[j]
    
    #parent directory for PGA and PGV 
    parent_directory_pga = nshmp_hazard_loc+rp_path_pga    ######You may wanna modify this to your scenario
    
    
    filespath_pga = []
    im_filepath_pga = []
    

    for i in range(len(site_name_single)):
        filespath_pga.append(parent_directory_pga+'/'+site_name_single[i]+'/data.csv')
        
        im_filepath_pga.append(parent_directory_pga+'/'+site_name_single[i]+'/summary.txt')
        
    target_pga = []

    #For target PGA

    for path in im_filepath_pga:
        with open(path, 'r', encoding='utf-8') as file:
            for line_number, line in enumerate(file, 1):
                if line_number == 7:
                    target_pga.append(float(line.strip().split()[-2]))
                    break


    #For bin data of PGA
    for i in range(len(filespath_pga)):
        temp = pd.read_csv(filespath_pga[i])
        
        if len(temp[temp[temp.columns[0]]>max_d].index)>0:
            index_to_remove = temp[temp[temp.columns[0]]>max_d].index[0]
            temp = temp.loc[:index_to_remove]

        temp[temp.columns[5]] = pd.to_numeric(temp[temp.columns[5]], errors='coerce')
        temp.dropna(subset=[temp.columns[5]], inplace=True)

        IM_level.extend([target_pga[i]]*len(temp))
        SiteName.extend([site_name_single[i]]*len(temp))
        R_distance.extend(temp[temp.columns[0]].tolist())
        Magnitude.extend(temp[temp.columns[2]].tolist())
        HazardContribution.extend(temp[temp.columns[5]].tolist())
        Epsilon_bar.extend(temp[temp.columns[4]].tolist())
        IM_type.extend(['PGA']*len(temp))
        ReturnPeriod.extend([RP]*len(temp))
        AnnualExceedanceRate.extend([1/RP]*len(temp))


Y_Deagg = pd.DataFrame(list(zip(ReturnPeriod, AnnualExceedanceRate, IM_level, IM_type, SiteName, R_distance, Magnitude,HazardContribution,
                               Epsilon_bar)), 
                       columns=['ReturnPeriod', 'AnnualExceedanceRate','IM_level', 'IM_type', 'SiteName', 'R_distance', 'Magnitude',
                               'HazardContribution', 'Epsilon_bar'])
Y_Deagg

,ReturnPeriod,AnnualExceedanceRate,IM_level,IM_type,SiteName,R_distance,Magnitude,HazardContribution,Epsilon_bar
0,10,0.100000,0.048430,PGA,Delta Site1,10.0,5.1,3.85,-0.81
1,10,0.100000,0.048430,PGA,Delta Site1,10.0,5.3,3.21,-1.35
2,10,0.100000,0.048430,PGA,Delta Site1,10.0,5.5,2.44,-1.92
3,10,0.100000,0.048430,PGA,Delta Site1,10.0,5.7,1.61,-2.13
4,10,0.100000,0.048430,PGA,Delta Site1,10.0,5.9,1.02,-2.35
...,...,...,...,...,...,...,...,...,...
74981,2475,0.000404,0.524751,PGA,Delta Site157,70.0,7.9,0.05,2.36
74982,2475,0.000404,0.524751,PGA,Delta Site157,70.0,8.1,0.10,2.16
74983,2475,0.000404,0.524751,PGA,Delta Site157,90.0,7.9,0.14,2.83
74984,2475,0.000404,0.524751,PGA,Delta Site157,90.0,8.1,0.43,2.63


In [14]:
Y_Deagg.to_csv(nshmp_hazard_loc+'/PGA_ydeagg.csv')

# Pre-select events based on deaggregation result

In [71]:
fault_candid_bin = Y_Deagg[(Y_Deagg['HazardContribution'] >= fault_threshold)]
fault_threshold

4

In [75]:
fault_candid_bin_reduced = fault_candid_bin.iloc[0:3]
fault_candid_bin

,ReturnPeriod,AnnualExceedanceRate,IM_level,IM_type,SiteName,R_distance,Magnitude,HazardContribution,Epsilon_bar
136,10,0.100000,0.050420,PGA,Delta Site2,30.0,5.3,4.15,-0.16
137,10,0.100000,0.050420,PGA,Delta Site2,30.0,5.5,4.03,-0.70
256,10,0.100000,0.045730,PGA,Delta Site3,30.0,5.3,4.08,-0.25
377,10,0.100000,0.047188,PGA,Delta Site4,30.0,5.3,4.08,-0.22
498,10,0.100000,0.049280,PGA,Delta Site5,30.0,5.3,4.17,-0.14
...,...,...,...,...,...,...,...,...,...
74947,2475,0.000404,0.524751,PGA,Delta Site157,10.0,5.9,7.36,1.03
74948,2475,0.000404,0.524751,PGA,Delta Site157,10.0,6.1,8.60,0.89
74949,2475,0.000404,0.524751,PGA,Delta Site157,10.0,6.3,10.17,1.03
74950,2475,0.000404,0.524751,PGA,Delta Site157,10.0,6.5,7.78,0.94


In [79]:
fault_candid_bin

,ReturnPeriod,AnnualExceedanceRate,IM_level,IM_type,SiteName,R_distance,Magnitude,HazardContribution,Epsilon_bar
136,10,0.100000,0.050420,PGA,Delta Site2,30.0,5.3,4.15,-0.16
137,10,0.100000,0.050420,PGA,Delta Site2,30.0,5.5,4.03,-0.70
256,10,0.100000,0.045730,PGA,Delta Site3,30.0,5.3,4.08,-0.25
377,10,0.100000,0.047188,PGA,Delta Site4,30.0,5.3,4.08,-0.22
498,10,0.100000,0.049280,PGA,Delta Site5,30.0,5.3,4.17,-0.14
...,...,...,...,...,...,...,...,...,...
74947,2475,0.000404,0.524751,PGA,Delta Site157,10.0,5.9,7.36,1.03
74948,2475,0.000404,0.524751,PGA,Delta Site157,10.0,6.1,8.60,0.89
74949,2475,0.000404,0.524751,PGA,Delta Site157,10.0,6.3,10.17,1.03
74950,2475,0.000404,0.524751,PGA,Delta Site157,10.0,6.5,7.78,0.94


In [53]:
fault_candid_bin_reduced = fault_candid_bin.iloc[0:1]

# get the range of mag and dist with hazard contribtion larget than a threshold
fm31_candid_f = pd.DataFrame(columns=flt_fm31f.columns) # create an empty dataframe


for i,row in fault_candid_bin_reduced.iterrows():
    ll_r = row['R_distance']-10
    ul_r = row['R_distance']+10
    ll_m = row['Magnitude']-0.1
    ul_m = row['Magnitude']+0.1
    
    # from flt_fm31f, retreive events that fall in the above range
    candid = flt_fm31f[(flt_fm31f['Rjb']>=ll_r) & (flt_fm31f['Rjb']<=ul_r) & (flt_fm31f['Mag']>=ll_m) & (flt_fm31f['Mag']<=ul_m)]
    print(candid)

Empty DataFrame
Columns: [Index, Mag, Rate, Depth, Dip, Rake, Width, Rrup, Rjb, ID]
Index: []


In [81]:
flt_fm31f

,Index,Mag,Rate,Depth,Dip,Rake,Width,Rrup,Rjb,ID
1,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.676,9.614238e-09,0.850,80.8,165.1,13.865,195.407038,195.407038,5827
2,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.677,1.253341e-08,0.874,80.8,165.1,13.834,190.118244,190.118244,5828
3,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.678,6.824738e-09,0.898,80.8,165.2,13.803,184.861774,184.861774,5829
4,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.679,1.230271e-08,0.925,80.8,165.2,13.768,179.640498,179.640498,5830
5,"1610:1607,1775:1772,1786:1779,1787:1797,1920:1...",7.680,7.191361e-09,0.959,80.9,165.3,13.724,174.491278,174.491278,5831
...,...,...,...,...,...,...,...,...,...,...
178394,2473:2475,6.496,4.425308e-07,1.200,60.0,-90.0,12.471,138.823718,138.823718,253605
178395,2473:2476,6.637,7.033105e-06,1.200,60.0,-90.0,12.471,138.823718,138.823718,253606
178396,2474:2475,6.303,6.821792e-07,1.200,60.0,-90.0,12.471,144.939447,144.939447,253607
178397,2474:2476,6.496,1.516271e-05,1.200,60.0,-90.0,12.471,144.939447,144.939447,253608


In [41]:
###threshold set up
fault_candid_bin = Y_Deagg[(Y_Deagg['HazardContribution'] >= fault_threshold)]
point_candid_bin = Y_Deagg[(Y_Deagg['HazardContribution'] >= point_threshold)]

fm31_candid_f = pd.DataFrame(columns=flt_fm31f.columns) # create an empty dataframe
fm31_candid_p = pd.DataFrame(columns=flt_fm31p.columns)
fm32_candid_f = pd.DataFrame(columns=flt_fm32f.columns)
fm32_candid_p = pd.DataFrame(columns=flt_fm31p.columns)

###Select rupture sources
for i, row in fault_candid_bin.iterrows(): # for each of the items in the rupture segment candidates
    ll_r = row['R_distance']-10
    ul_r = row['R_distance']+10
    ll_m = row['Magnitude']-0.1
    ul_m = row['Magnitude']+0.1
    
    candid = flt_fm31f[(flt_fm31f['Rjb']>=ll_r) & (flt_fm31f['Rjb']<=ul_r) & (flt_fm31f['Mag']>=ll_m) & (flt_fm31f['Mag']<=ul_m)]
    if len(candid)>0:
        random_row = candid.sample(n=1)
        fm31_candid_f = pd.concat([fm31_candid_f,random_row], ignore_index=True)
        
    candid = flt_fm32f[(flt_fm32f['Rjb']>=ll_r) & (flt_fm32f['Rjb']<=ul_r) & (flt_fm32f['Mag']>=ll_m) & (flt_fm32f['Mag']<=ul_m)]
    if len(candid)>0:
        random_row = candid.sample(n=1)
        fm32_candid_f = pd.concat([fm32_candid_f,random_row], ignore_index=True)

###Select POINT sources
for i, row in point_candid_bin.iterrows():
    ll_r = row['R_distance']-10
    ul_r = row['R_distance']+10
    ll_m = row['Magnitude']-0.1
    ul_m = row['Magnitude']+0.1
    
    candid = flt_fm31p[(flt_fm31p['Rjb']>=ll_r) & (flt_fm31p['Rjb']<=ul_r) & (flt_fm31p['Mag']>=ll_m) & (flt_fm31p['Mag']<=ul_m)]
    if len(candid)>0:
        random_row = candid.sample(n=1)
        fm31_candid_p = pd.concat([fm31_candid_p,random_row], ignore_index=True)
        
    candid = flt_fm32p[(flt_fm32p['Rjb']>=ll_r) & (flt_fm32p['Rjb']<=ul_r) & (flt_fm32p['Mag']>=ll_m) & (flt_fm32p['Mag']<=ul_m)]
    if len(candid)>0:
        random_row = candid.sample(n=1)
        fm32_candid_p = pd.concat([fm32_candid_p,random_row], ignore_index=True)

fm31_candid_f = fm31_candid_f.drop_duplicates(subset=['ID'], keep='first')
fm31_candid_f.reset_index(drop=True, inplace=True)

fm32_candid_f = fm32_candid_f.drop_duplicates(subset=['ID'], keep='first')
fm32_candid_f.reset_index(drop=True, inplace=True)


fm31_candid_p = fm31_candid_p.drop_duplicates(subset=['ID'], keep='first')
fm31_candid_p.reset_index(drop=True, inplace=True)

fm32_candid_p = fm32_candid_p.drop_duplicates(subset=['ID'], keep='first')
fm32_candid_p.reset_index(drop=True, inplace=True)

print('Number of candidate rupture sources',len(fm32_candid_f)+len(fm31_candid_f), 'out of the original filtered source', len(flt_fm31f) + len(flt_fm32f) )
print('Number of candidate point sources',len(fm32_candid_p)+len(fm31_candid_p), 'out of the original filtered source', len(flt_fm31p) + len(flt_fm32p) )

Number of candidate rupture sources 969 out of the original filtered source 397886
Number of candidate point sources 799 out of the original filtered source 220980


In [92]:
#### Event set file
Fault_type = {1.0: 'SS', 2.0:'N', 3.0:'R'}

###FM31 Point Sources
col_num = len(sites)
row_num = len(fm31_candid_p)
info_mat3 = np.zeros([row_num, 4] , dtype=object)

for i, row1 in fm31_candid_p.iterrows():
    info_mat3[i][0] = 'FM31_'+ str(row1['Longitude'])+','+str(row1['Latitude'])+',0.0'+'_'+Fault_type[row1['Fault_Type']]
    info_mat3[i][1] = row1['Mag']
    info_mat3[i][2] = row1['Rate']
    info_mat3[i][3] = Fault_type[row1['Fault_Type']]
        
        
###FM32 Point Sources
col_num = len(sites)
row_num = len(fm32_candid_p)
info_mat4 = np.zeros([row_num, 4] , dtype=object)

for i, row1 in fm32_candid_p.iterrows():
    info_mat4[i][0] = 'FM32_'+ str(row1['Longitude'])+','+str(row1['Latitude'])+',0.0'+'_'+Fault_type[row1['Fault_Type']]
    info_mat4[i][1] = row1['Mag']
    info_mat4[i][2] = row1['Rate']
    info_mat4[i][3] = Fault_type[row1['Fault_Type']]
        
        
###FM 31 Ruptures sources
col_num = len(sites)
row_num = len(fm31_candid_f)
info_mat1 = np.zeros([row_num, 4] , dtype=object)

for i in range(len(fm31_candid_f)):
    info_mat1[i][0] = 'FM31_'+fm31_candid_f.iloc[i][0]
    info_mat1[i][1] = fm31_candid_f.iloc[i][1]
    info_mat1[i][2] = fm31_candid_f.iloc[i][2]
    info_mat1[i][3] = 'SS'
        
        
###FM 32 Ruptures sources
col_num = len(sites)
row_num = len(fm32_candid_f)
info_mat2 = np.zeros([row_num, 4] , dtype=object)

for i in range(len(fm32_candid_f)):
    info_mat2[i][0] = 'FM32_'+fm32_candid_f.iloc[i][0]
    info_mat2[i][1] = fm32_candid_f.iloc[i][1]
    info_mat2[i][2] = fm32_candid_f.iloc[i][2]
    info_mat2[i][3] = 'SS'


In [93]:
#FM31_Rups['ID'] = FM31_Rups['ID'].apply(lambda arr: arr.item())
#FM32_Rups['ID'] = FM32_Rups['ID'].apply(lambda arr: arr.item())

In [94]:
fm31_indices = []
fm32_indices = []
####organize subsection information for potential eventsets
for i in range(len(fm31_candid_f)):
    index = FM31_Rups.index[FM31_Rups['ID'] == fm31_candid_f.iloc[i][9]][0]
    fm31_indices.append(FM31_Rups.iloc[index][1])
fm31_indices  

for i in range(len(fm32_candid_f)):
    index = FM32_Rups.index[FM32_Rups['ID'] == fm32_candid_f.iloc[i][9]][0]
    fm32_indices.append(FM32_Rups.iloc[index][1])
#fm32_indices  

In [5]:
####Convert RrupRjb function to tbe callable in python and rename it as rRrupRjb
rRrupRjb = robjects.r['RrupRjb']

In [30]:
## temporiary
#Great Valley 06 (Mildland)
x1 = -121.8201
y1 = 38.4287

x2 = -121.7411
y2 = 38.3137

x3 = -121.7088
y3 = 38.2545

x4 = -121.6574
y4 = 38.1672

x5 = -121.6343
y5 = 38.0829

x6 = -121.6172
y6 = 37.9982

x7 = -121.6294
y7 = 37.9071

x8 = -121.6481
y8 = 37.8457

rRes = rRrupRjb(fflat1 = y1, fflon1 = x1, 
                        fflat2 = y8, fflon2 = x8, 
                        fdip = 47, topd = 15, width = 100,
                        slon = -121.4458, slat = 37.9412)


In [31]:
Res = dict(zip(rRes.names, map(list, list(rRes))))
Res

{'Rrup': [24.787216917263923], 'Rjb': [19.733375851675625]}

In [32]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()

revents_hazmat_calc = robjects.r['events_hazmat_calc']
rscenario_selection = robjects.r['scenario_selection']
rflts_filter = robjects.r['flts_filter']
rpts_filter = robjects.r['pts_filter']

In [38]:
a = rflts_filter( 37.8457,-121.6481, max_dist = 2, branch = 1) 
aa = ro.conversion.get_conversion().rpy2py(a)
aa

,Index,Mag,Rate,Depth,Dip,Rake,Width,Rrup,Rjb,ID
1,716:721,7.082,0.000013,2.4,47.0,90.0,17.228,1.000003,0.00062,100937
2,715:721,7.162,0.000008,2.4,47.0,90.0,17.228,1.000003,0.00062,100943
3,714:721,7.232,0.000019,2.4,47.0,90.0,17.228,1.000003,0.00062,100948
4,717:721,6.987,0.000015,2.4,47.0,90.0,17.228,1.000003,0.00062,100952
5,718:721,6.871,0.000026,2.4,47.0,90.0,17.228,1.000003,0.00062,100955
6,719:721,6.726,0.000046,2.4,47.0,90.0,17.228,1.000003,0.00062,100957
7,720:721,6.539,0.000144,2.4,47.0,90.0,17.228,1.000003,0.00062,100958


In [96]:
###Calculate Rjb for eventset to sites
row_num = len(fm31_candid_f)
col_num = len(sites)
dist_mat1 = np.zeros([row_num, col_num])

sites_lon = np.array(sites_x, dtype=float)
sites_lat = np.array(sites_y, dtype=float)

# Convert NumPy arrays to R vectors
slon_r = robjects.vectors.FloatVector(sites_lon)
slat_r = robjects.vectors.FloatVector(sites_lat)

for i, row1 in fm31_candid_f.iterrows():
    trace = row1['Index']
    range_strings = trace.split(',')

    rjb_list = []
    for j in fm31_indices[i]:

        rRes = rRrupRjb(fflat1 = FM31_BeEn_Trace[j][0][1], fflon1 = FM31_BeEn_Trace[j][0][0], 
                        fflat2 = FM31_BeEn_Trace[j][1][1], fflon2 = FM31_BeEn_Trace[j][1][0], 
                        fdip = row1['Dip'], topd = row1['Depth'], 
                        width = row1['Width'],slon = slon_r, slat = slat_r)

        Res = dict(zip(rRes.names, map(list, list(rRes))))
        rjb_list.append(Res['Rjb'])

    # Calculate min Rjb from each segments
    dist_mat1[i] = np.min(rjb_list, axis=0)

print(np.shape(dist_mat1))

row_num = len(fm32_candid_f)
dist_mat2 = np.zeros([row_num, col_num])

for i, row1 in fm32_candid_f.iterrows():
    trace = row1['Index']
    range_strings = trace.split(',')

    rjb_list = []
    for j in fm32_indices[i]:

        rRes = rRrupRjb(fflat1 = FM32_BeEn_Trace[j][0][1], fflon1 = FM32_BeEn_Trace[j][0][0], 
                        fflat2 = FM32_BeEn_Trace[j][1][1], fflon2 = FM32_BeEn_Trace[j][1][0], 
                        fdip = row1['Dip'], topd = row1['Depth'], 
                        width = row1['Width'],slon = slon_r, slat = slat_r)

        Res = dict(zip(rRes.names, map(list, list(rRes))))
        rjb_list.append(Res['Rjb'])

    # Calculate min Rjb from each segments
    dist_mat2[i] = np.min(rjb_list, axis=0)

print(np.shape(dist_mat2))

(446, 157)
(517, 157)


In [98]:
###FM31 Point Sources

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the haversine distance between two points (lon1, lat1) and (lon2, lat2) in kilometers.
    """
    # Convert latitude and longitude from degrees to radians
    lon1, lat1, lon2, lat2 = np.radians([lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    radius = 6371  # Earth's radius in kilometers
    distance = radius * c

    return distance

col_num = len(sites)
row_num = len(fm31_candid_p)
dist_mat3 = np.zeros([row_num, col_num])

for i, row1 in fm31_candid_p.iterrows():
  
    for j, row2 in sites.iterrows():
        dist_mat3[i][j] = haversine_distance(row1['Longitude'], row1['Latitude'], row2['X'], row2['Y'])
        
        
###FM32 Point Sources

col_num = len(sites)
row_num = len(fm32_candid_p)
dist_mat4 = np.zeros([row_num, col_num])

for i, row1 in fm32_candid_p.iterrows():

    for j, row2 in sites.iterrows():
        dist_mat4[i][j] = haversine_distance(row1['Longitude'], row1['Latitude'], row2['X'], row2['Y'])
        

In [99]:
####Concatenate results
mat1 = np.concatenate((info_mat1, dist_mat1), axis = 1)
mat2 = np.concatenate((info_mat2, dist_mat2), axis = 1)
mat3 = np.concatenate((info_mat3, dist_mat3), axis = 1)
mat4 = np.concatenate((info_mat4, dist_mat4), axis = 1)

mat_t = np.concatenate((mat1, mat2,mat3,mat4), axis = 0)


####Construct Event Set file

column_name = []

for i in range(len(sites)):
    column_name.append('Rjb to Site'+str(i+1))
column_name = ['EventID', 'Magnitude', 'AnnualOccurrenceRate','Fault_type'] + column_name

EventSet = pd.DataFrame(data = mat_t,  columns = column_name)
EventSet # Combine rup and point sources from truncated haz. disagg pre-selection associated with their information and r_jb to all sites

,EventID,Magnitude,AnnualOccurrenceRate,Fault_type,Rjb to Site1,Rjb to Site2,Rjb to Site3,Rjb to Site4,Rjb to Site5,Rjb to Site6,...,Rjb to Site148,Rjb to Site149,Rjb to Site150,Rjb to Site151,Rjb to Site152,Rjb to Site153,Rjb to Site154,Rjb to Site155,Rjb to Site156,Rjb to Site157
0,"FM31_831:835,818:819",6.785,0.000048,SS,34.254724,31.231534,40.402213,37.928738,34.928908,64.801718,...,65.764187,62.911133,60.888997,58.736544,56.051075,64.094014,62.055004,60.740408,66.233918,65.006103
1,FM31_831:835,6.664,0.000185,SS,36.014524,31.848409,42.224375,38.890614,35.069948,69.562004,...,65.452048,62.599451,60.579036,58.430878,55.760327,63.785003,61.750647,60.441576,65.93115,64.708923
2,"FM31_202,201",6.167,0.00004,SS,26.072661,21.377721,32.122992,28.449617,24.221725,60.79516,...,58.532869,57.469683,56.995159,56.885747,58.041153,60.511207,60.410466,60.696181,64.816528,65.085182
3,FM31_728:731,6.101,0.00001,SS,70.007645,65.726672,70.699493,67.217989,62.932701,94.78437,...,32.359771,26.887994,22.53805,17.16542,8.118694,23.560947,18.488858,14.682079,20.877258,17.596476
4,FM31_728:736,6.5,0.000003,SS,70.007645,65.726672,70.699493,67.217989,62.932701,94.78437,...,32.359771,26.887994,22.53805,17.16542,8.118694,23.560947,18.488858,14.682079,20.877258,17.596476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1760,"FM32_-122.2,37.2,0.0_R",7.85,0.0,R,98.138841,92.798976,103.650053,99.414754,94.148193,133.461942,...,108.206341,104.476173,101.612533,98.198974,92.580224,104.363699,101.045014,98.535549,104.661195,102.242483
1761,"FM32_-120.8,38.9,0.0_N",7.85,0.0,N,134.544096,138.529026,128.30411,131.482053,135.554962,106.312479,...,122.854066,128.058218,132.242377,137.495851,147.008875,131.110266,136.406602,140.658308,135.159444,139.448518
1762,"FM32_-120.3,37.7,0.0_SS",7.85,0.0,SS,145.725742,144.38692,142.753589,141.501568,140.119442,151.250406,...,90.747966,89.784453,89.255815,88.904032,89.145357,85.756792,85.388172,85.353259,80.993656,80.95441
1763,"FM32_-121.3,39.1,0.0_N",7.85,0.0,N,127.874804,132.977445,122.054881,126.164466,131.332437,93.422252,...,136.772529,142.332169,146.779887,152.339541,162.346933,146.800554,152.359468,156.806641,152.496395,156.939761


In [100]:
EventSet['Magnitude'] = EventSet['Magnitude'].apply(lambda arr: arr.item())
EventSet['AnnualOccurrenceRate'] = EventSet['AnnualOccurrenceRate'].apply(lambda arr: arr.item())

Y_Haz['IM_level'] = Y_Haz['IM_level'].astype(float)
#Y_Haz.iloc[0][1]

In [101]:
for i in range(1, len(sites)+1):
    name = 'Rjb to Site'+str(i)
    EventSet[name] = EventSet[name].astype(float)

In [102]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

with (ro.default_converter + pandas2ri.converter).context():
    Y_Haz_r = ro.conversion.get_conversion().py2rpy(Y_Haz)
    
with (ro.default_converter + pandas2ri.converter).context():
    SiteTable_r = ro.conversion.get_conversion().py2rpy(SiteTable)
    
with (ro.default_converter + pandas2ri.converter).context():
    EventSet_r = ro.conversion.get_conversion().py2rpy(EventSet)

EventSet_r # add more infomration to the candidate sources

EventID,Magnitude,AnnualOccurrenceRate,...,Rjb to Site155,Rjb to Site156,Rjb to Site157
...,...,...,...,...,...,...


In [119]:
Y_Haz.to_csv('output/Y_Haz_zehan.csv')
EventSet.to_csv('output/EventSet_zehan.csv')
SiteTable.to_csv('output/SiteTable_zehan.csv')

# Store the important variables for plotting
import pickle 

#with open('rphsadata_1002.pkl', 'wb') as file:
#    pickle.dump((fm31_candid_f,fm32_candid_f,fm31_candid_p,fm32_candid_p,fm31_indices,fm32_indices,errors_mat,PGA_val,target_y,predict_y), file)


with open('rphsadata_zehan.pkl', 'wb') as file:
    pickle.dump((fm32_candid_f, fm31_candid_f, fm32_candid_p, fm31_candid_p, flt_fm31f, flt_fm32f, flt_fm31p, flt_fm32p, fm31_indices, fm32_indices,dist_mat1,dist_mat2,dist_mat3,dist_mat4,Y_Haz, EventSet, SiteTable), file)


In [104]:
X_haz = revents_hazmat_calc(Y_haz = Y_Haz_r, EventSet = EventSet_r, SiteTable = SiteTable_r)

[1] "The calculation is completed by 0%"
[1] "The calculation is completed by 1%"
[1] "The calculation is completed by 2%"
[1] "The calculation is completed by 3%"
[1] "The calculation is completed by 4%"
[1] "The calculation is completed by 5%"
[1] "The calculation is completed by 6%"
[1] "The calculation is completed by 7%"
[1] "The calculation is completed by 8%"
[1] "The calculation is completed by 9%"
[1] "The calculation is completed by 10%"
[1] "The calculation is completed by 11%"
[1] "The calculation is completed by 12%"
[1] "The calculation is completed by 13%"
[1] "The calculation is completed by 14%"
[1] "The calculation is completed by 15%"
[1] "The calculation is completed by 16%"
[1] "The calculation is completed by 17%"
[1] "The calculation is completed by 18%"
[1] "The calculation is completed by 19%"
[1] "The calculation is completed by 20%"
[1] "The calculation is completed by 21%"
[1] "The calculation is completed by 22%"
[1] "The calculation is completed by 23%"
[1

In [105]:
X_haz_p = ro.conversion.rpy2py(X_haz)
X_haz_p

,IM_type,IM_level,SiteName,"EventID_FM31_831:835,818:819",EventID_FM31_831:835,"EventID_FM31_202,201",EventID_FM31_728:731,EventID_FM31_728:736,EventID_FM31_205:203,EventID_FM31_724:727,...,"EventID_FM32_-122.5,38.6,0.0_SS","EventID_FM32_-121.3,37.2,0.0_R","EventID_FM32_-122.3,37.2,0.0_N","EventID_FM32_-122.8,38.2,0.0_SS","EventID_FM32_-121.0,37.4,0.0_R","EventID_FM32_-122.2,37.2,0.0_R","EventID_FM32_-120.8,38.9,0.0_N","EventID_FM32_-120.3,37.7,0.0_SS","EventID_FM32_-121.3,39.1,0.0_N","EventID_FM32_-120.7,38.4,0.0_R"
785,PGA,0.0253,Delta Site1,4.735879e-05,1.836164e-04,3.952618e-05,7.713522e-06,2.267099e-06,2.354136e-05,1.349523e-06,...,3.775160e-09,0.0,0.0,7.517689e-10,2.075795e-08,5.453402e-12,6.272779e-10,3.692314e-10,1.563482e-10,1.885245e-08
786,PGA,0.0253,Delta Site2,4.742829e-05,1.841657e-04,3.964726e-05,8.135245e-06,2.335486e-06,2.375130e-05,1.393044e-06,...,3.762278e-09,0.0,0.0,7.510122e-10,2.104998e-08,5.496488e-12,6.084674e-10,3.721299e-10,1.509692e-10,1.857891e-08
787,PGA,0.0253,Delta Site3,4.714288e-05,1.823396e-04,3.922476e-05,7.642722e-06,2.255295e-06,2.323055e-05,1.341703e-06,...,3.776007e-09,0.0,0.0,7.497264e-10,2.076462e-08,5.399962e-12,6.552339e-10,3.755962e-10,1.620734e-10,1.951760e-08
788,PGA,0.0253,Delta Site4,4.724328e-05,1.831011e-04,3.943051e-05,7.991729e-06,2.312583e-06,2.349648e-05,1.378693e-06,...,3.766758e-09,0.0,0.0,7.492248e-10,2.101599e-08,5.441875e-12,6.412392e-10,3.782002e-10,1.580781e-10,1.934499e-08
789,PGA,0.0253,Delta Site5,4.734032e-05,1.837600e-04,3.958352e-05,8.393267e-06,2.375696e-06,2.371915e-05,1.417847e-06,...,3.752708e-09,0.0,0.0,7.483601e-10,2.129348e-08,5.486359e-12,6.225722e-10,3.810203e-10,1.527376e-10,1.909578e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,PGA,1.4600,Delta Site153,6.856277e-12,1.466991e-11,3.578745e-13,2.867199e-10,2.243037e-10,4.153623e-13,1.012634e-10,...,9.307116e-17,0.0,0.0,1.728981e-17,7.425933e-13,1.538802e-18,4.616324e-18,1.076720e-15,3.636325e-19,3.003542e-15
2508,PGA,1.4600,Delta Site154,9.473375e-12,2.055243e-11,3.650873e-13,1.216630e-09,7.805318e-10,6.155198e-13,2.883976e-10,...,7.247031e-17,0.0,0.0,1.512866e-17,1.353531e-12,2.175578e-18,3.181223e-18,1.118206e-15,2.452964e-19,1.691000e-15
2509,PGA,1.4600,Delta Site155,1.168111e-11,2.556145e-11,3.450067e-13,3.964593e-09,2.163771e-09,7.877909e-13,7.051725e-10,...,5.896447e-17,0.0,0.0,1.347160e-17,2.169532e-12,2.825508e-18,2.357950e-18,1.122217e-15,1.788664e-19,1.058407e-15
2510,PGA,1.4600,Delta Site156,4.892929e-12,1.030108e-11,1.535809e-13,6.062123e-10,4.279402e-10,2.654560e-13,1.047448e-10,...,5.798819e-17,0.0,0.0,1.145396e-17,1.803541e-12,1.491713e-18,3.472966e-18,1.754980e-15,2.429255e-19,2.274255e-15


In [107]:
col_names = ['AnnualExceedanceRate']  # Indices of columns you want to extract
Y = Y_Haz[col_names].values

excluded_columns = ['IM_type','IM_level', 'SiteName']

# Use .loc to extract all rows and columns except excluded columns
result_df = X_haz_p.loc[:, ~X_haz_p.columns.isin(excluded_columns)]
X = result_df.values
X

array([[4.73587856e-05, 1.83616370e-04, 3.95261847e-05, ...,
        3.69231394e-10, 1.56348219e-10, 1.88524478e-08],
       [4.74282918e-05, 1.84165654e-04, 3.96472645e-05, ...,
        3.72129937e-10, 1.50969164e-10, 1.85789139e-08],
       [4.71428767e-05, 1.82339647e-04, 3.92247566e-05, ...,
        3.75596177e-10, 1.62073435e-10, 1.95175986e-08],
       ...,
       [1.16811126e-11, 2.55614542e-11, 3.45006744e-13, ...,
        1.12221736e-15, 1.78866434e-19, 1.05840692e-15],
       [4.89292875e-12, 1.03010784e-11, 1.53580947e-13, ...,
        1.75498031e-15, 2.42925497e-19, 2.27425510e-15],
       [5.93657837e-12, 1.25954300e-11, 1.45744678e-13, ...,
        1.76206049e-15, 1.77181203e-19, 1.40601405e-15]])

In [108]:
Y_r = robjects.vectors.FloatVector(Y)
Y_r

import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

nr,nc = X.shape
X_r = ro.r.matrix(X, nrow=nr, ncol=nc)
X_r

array([[4.73587856e-05, 1.83616370e-04, 3.95261847e-05, ...,
        3.69231394e-10, 1.56348219e-10, 1.88524478e-08],
       [4.74282918e-05, 1.84165654e-04, 3.96472645e-05, ...,
        3.72129937e-10, 1.50969164e-10, 1.85789139e-08],
       [4.71428767e-05, 1.82339647e-04, 3.92247566e-05, ...,
        3.75596177e-10, 1.62073435e-10, 1.95175986e-08],
       ...,
       [1.16811126e-11, 2.55614542e-11, 3.45006744e-13, ...,
        1.12221736e-15, 1.78866434e-19, 1.05840692e-15],
       [4.89292875e-12, 1.03010784e-11, 1.53580947e-13, ...,
        1.75498031e-15, 2.42925497e-19, 2.27425510e-15],
       [5.93657837e-12, 1.25954300e-11, 1.45744678e-13, ...,
        1.76206049e-15, 1.77181203e-19, 1.40601405e-15]])

In [109]:
#max_rate_multiplier=10000
res = rscenario_selection(Y = Y_r, X = X_r, num_lambda = 10000)

Conduct selection 
  |======================================================================| 100%
Relaxed Fits
  |======================================================================| 100%
Prepare and output results 
  |======================================================================| 100%
 The results are not written into any files. 


In [110]:
pga_res = dict(zip(res.names, map(list, list(res))))
beta_mat = pga_res['betas']
yhat_mat = pga_res['Y_hat']
errors_mat = pga_res['errors_mat']
#beta_mat

In [111]:
np.shape(beta_mat)

(1765, 38)

In [112]:
#mean of squared logarithmic residuals, mean of absolute logarithmic relative errors, 
#and mean of absolute arithmetic relative errors
errors_mat

[array([ 1.        , 15.31594385,  0.3222717 ,  0.58808632]),
 array([2.        , 4.63564345, 0.16592854, 0.47503441]),
 array([3.        , 3.88101963, 0.14376167, 0.43810448]),
 array([4.        , 1.94645331, 0.08963433, 0.33348096]),
 array([5.        , 1.77427004, 0.0800684 , 0.31020503]),
 array([6.        , 3.30329617, 0.12199225, 0.39128841]),
 array([7.        , 1.40082201, 0.06783205, 0.28066749]),
 array([8.        , 1.43648201, 0.06842851, 0.28205279]),
 array([9.        , 0.88037848, 0.05134941, 0.23930018]),
 array([10.        ,  0.82674231,  0.0491469 ,  0.23137134]),
 array([11.        ,  0.71639276,  0.04495485,  0.21936857]),
 array([12.        ,  0.71659133,  0.04496297,  0.21939687]),
 array([13.        ,  0.30960579,  0.03026612,  0.17925144]),
 array([14.        ,  0.3149224 ,  0.03001249,  0.17982566]),
 array([15.        ,  0.27072663,  0.02836054,  0.17410535]),
 array([16.        ,  0.25458518,  0.02679531,  0.16767201]),
 array([17.        ,  0.25254316,  0.026

In [113]:
PGA_val = []
target_y = []
predict_y = []

for i in range(len(Y_Haz)//len(sites)):
    PGA_val.append(Y_Haz.iloc[i*len(sites)][1])

for j in example_site_index:
    for i in range(len(Y_Haz)//len(sites)):
        target_y.append(Y_Haz.iloc[i*len(sites)+j][3])
        
for i in example_site_index:
    for j in range(len(Y_Haz)//len(sites)):
        predict_y.append(yhat_mat[j*len(sites)+i][21])

In [129]:
yhat_mat[1580]

array([9.48209882e-10, 1.20891491e-07, 1.71616686e-07, 1.93988762e-07,
       3.76185990e-07, 1.48705936e-07, 5.61171300e-07, 5.57334674e-07,
       3.02154510e-06, 3.40160663e-06, 4.97966511e-06, 4.97841826e-06,
       4.35645035e-06, 4.56152719e-06, 3.98328772e-06, 4.45682223e-06,
       4.46639576e-06, 9.53150820e-06, 9.40388350e-06, 9.05507976e-06,
       9.95678645e-06, 1.03062077e-05, 8.20790385e-06, 8.93944257e-06,
       1.11114779e-05, 9.43825667e-06, 7.77619799e-06, 9.93773348e-06,
       9.95874553e-06, 1.05391075e-05, 7.83297401e-06, 8.26558708e-06,
       9.19845560e-06, 1.14061641e-05, 9.60955084e-06, 9.60991770e-06,
       1.01247587e-05, 1.01793606e-05])

In [38]:
# Store the important variables for plotting
import pickle 

with open('rphsadata_1002.pkl', 'wb') as file:
    pickle.dump((errors_mat,PGA_val,target_y,predict_y), file)


In [ ]:
'''
get_ipython().run_line_magic('matplotlib', 'inline')
fig, ax = plt.subplots()

ax.set_xlabel('Number of Selected events')
ax.set_ylabel("Error metrics")
ax.set_yscale("log")
ax.set_title("PGA hazard matching")
x = [arr[0] for arr in errors_mat]  
y1= [arr[1] for arr in errors_mat]
y2= [arr[2] for arr in errors_mat]
y3= [arr[3] for arr in errors_mat]
ax.plot(x, y1, label='mean of squared logarithmic residuals')
ax.plot(x, y2, label='mean of absolute logarithmic relative errors')
ax.plot(x, y3, label='mean of absolute arithmetic relative errors')
ax.grid(True, alpha=0.5, which='both')
ax.legend(loc='upper right')
'''

In [ ]:
# get_ipython().run_line_magic('matplotlib', 'inline')
# fig, ax = plt.subplots()

# ax.set_xlabel('Number of Selected events')
# ax.set_ylabel("Error metrics")
# ax.set_title("PGA hazard matching")
# x = [arr[0] for arr in errors_mat]  
# #y1= [arr[1] for arr in errors_mat]
# y2= [arr[2] for arr in errors_mat]
# y3= [arr[3] for arr in errors_mat]
# #ax.plot(x, y1, label='mean of squared logarithmic residuals')
# ax.plot(x, y2, label='mean of absolute logarithmic relative errors')
# ax.plot(x, y3, label='mean of absolute arithmetic relative errors')
# ax.grid(True, alpha=0.5, which='both')
# ax.legend(loc='upper right')

In [ ]:
'''
get_ipython().run_line_magic('matplotlib', 'inline')
fig, ax = plt.subplots()

ax.set_xlabel('Number of Selected events')
ax.set_ylabel("Error metrics")
#ax.set_yscale("log")
ax.set_title("PGA hazard matching")
x = [arr[0] for arr in errors_mat]  
#y1= [arr[1] for arr in errors_mat]
#y2= [arr[2] for arr in errors_mat]
y3= [arr[3] for arr in errors_mat]
#ax.plot(x, y1, label='mean of squared logarithmic residuals')
#ax.plot(x, y2, label='mean of absolute logarithmic relative errors')
ax.plot(x, y3, label='mean of absolute arithmetic relative errors')
ax.grid(True, alpha=0.5, which='both')
ax.legend(loc='upper right')
'''

In [ ]:
'''
get_ipython().run_line_magic('matplotlib', 'inline')
fig, ax = plt.subplots(1, 3,sharey=True, sharex=True)

ax[0].set_xlabel('PGA (g)')
ax[0].set_ylabel("Annual Exceedance Rate")
ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[0].set_ylim([0.000404,0.1])
#ax[0].set_xlim([0.04,2])
ax[0].set_title('Site 1 in the Delta', fontsize="10")

ax[1].set_xlabel('PGA (g)')
ax[1].set_xscale("log")
ax[1].set_yscale("log")
#ax[1].set_xlim([0.02,1])
ax[1].set_title('Site 57 in the Delta',fontsize="10")

ax[2].set_xlabel('PGA (g)')
ax[2].set_xscale("log")
ax[2].set_yscale("log")
#ax[2].set_xlim([0.04,1])
ax[2].set_title('Site 155 in the Delta', fontsize="10")

ax[0].plot(PGA_val, target_y[:11] ,label='True hazard')
ax[0].plot(PGA_val, predict_y[:11] ,label='Predicted hazard',linestyle='dashed')
ax[1].plot(PGA_val, target_y[11:22] ,label='Target hazard curve')
ax[1].plot(PGA_val, predict_y[11:22] ,label='Predicted hazard',linestyle='dashed')
ax[2].plot(PGA_val, target_y[22:] ,label='Target hazard curve')
ax[2].plot(PGA_val, predict_y[22:] ,label='Predicted hazard',linestyle='dashed')

ax[0].grid(True, alpha=0.5, which='both')
ax[0].legend(fontsize="8")
ax[1].grid(True, alpha=0.5, which='both')
ax[1].legend(fontsize="8")
ax[2].grid(True, alpha=0.5, which='both')
ax[2].legend(fontsize="8")

fig.set_figheight(3.3)
fig.set_figwidth(10)
plt.tight_layout()
'''

In [39]:
np.shape(yhat_mat)

(1727, 23)

In [40]:
EventSet

,EventID,Magnitude,AnnualOccurrenceRate,Fault_type,Rjb to Site1,Rjb to Site2,Rjb to Site3,Rjb to Site4,Rjb to Site5,Rjb to Site6,...,Rjb to Site148,Rjb to Site149,Rjb to Site150,Rjb to Site151,Rjb to Site152,Rjb to Site153,Rjb to Site154,Rjb to Site155,Rjb to Site156,Rjb to Site157
0,FM31_726:739,6.717,1.213760e-05,SS,62.690983,58.411708,63.388749,59.899343,55.616121,87.745874,...,28.017030,22.880133,18.939092,14.426128,7.903481,20.942809,16.972889,14.153594,20.378653,17.561926
1,FM31_727:739,6.681,1.005274e-05,SS,66.348521,62.067552,67.044042,63.558824,59.273579,91.259620,...,30.044013,24.694597,20.491930,15.426522,7.903481,21.987468,17.365191,14.155579,20.397120,17.561926
2,"FM31_347:346,781:780",6.114,1.096194e-05,SS,14.639094,11.736244,20.707180,18.369047,15.744126,36.535129,...,62.861809,63.902520,65.064736,66.904506,71.193041,68.440860,70.194361,71.875388,74.336448,75.927946
3,FM31_725:728,6.101,1.247775e-05,SS,59.031448,54.752193,59.733859,56.239770,51.956589,84.244193,...,26.343911,21.538841,18.005804,14.249143,7.903481,20.506061,16.972886,14.153594,20.378653,17.561926
4,FM31_724:731,6.442,8.715094e-07,SS,55.373404,51.095332,56.079839,52.580436,48.298601,80.756865,...,25.096975,20.764074,17.774309,14.249143,7.903481,20.496184,16.972886,14.153594,20.378653,17.561926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1737,"FM32_-121.1,39.0,0.0_SS",7.850,3.858223e-10,SS,126.497790,131.188952,120.386882,124.158162,128.932914,94.141715,...,126.974497,132.479545,136.886766,142.399340,152.330613,136.464430,141.993124,146.417772,141.604530,146.040746
1738,"FM32_-120.3,37.7,0.0_R",7.850,4.504808e-10,R,145.725742,144.386920,142.753589,141.501568,140.119442,151.250406,...,90.747966,89.784453,89.255815,88.904032,89.145357,85.756792,85.388172,85.353259,80.993656,80.954410
1739,"FM32_-120.3,38.4,0.0_R",7.850,7.996524e-10,R,145.080310,146.715133,139.678494,140.877769,142.557908,131.695583,...,106.189099,109.396794,112.095469,115.620930,122.347986,109.356469,112.965617,115.964021,109.713287,112.796435
1740,"FM32_-121.1,37.4,0.0_SS",7.350,1.809642e-07,SS,100.867803,96.978905,101.169244,97.851190,93.835187,123.602957,...,55.436675,50.231904,46.127768,41.098282,32.483760,44.830435,39.635667,35.559417,38.188237,33.937689


In [117]:
####Extract the selected events
beta_selected_column = [arr[column_index] for arr in beta_mat]

idx_list = []
multipier_list = []
for i, val in enumerate(beta_selected_column):
    if val !=0:
        idx_list.append(i)
        multipier_list.append(val)
len(idx_list)

selected_events = EventSet.iloc[idx_list]
selected_events = selected_events.reset_index(drop=True)

AOR_N = []
for i, row in selected_events.iterrows():
    AOR_N.append(multipier_list[i]*row['AnnualOccurrenceRate'])
len(AOR_N)

PGA_selected_events = selected_events[['EventID', 'Magnitude', 'AnnualOccurrenceRate','Fault_type' ]]
#PGA_selected_events.loc[:, 'Adjusted AnnualOccurrenceRate'] = AOR_N
PGA_selected_events['Adjusted AnnualOccurrenceRate'] = AOR_N
PGA_selected_events

C:\Users\Chenhao\AppData\Local\Temp\ipykernel_14772\2450204455.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PGA_selected_events['Adjusted AnnualOccurrenceRate'] = AOR_N


,EventID,Magnitude,AnnualOccurrenceRate,Fault_type,Adjusted AnnualOccurrenceRate
0,"FM31_822:818,835:832,1283:1294",7.000,1.912041e-07,SS,0.019959
1,"FM31_1730:1723,827:818,835:832,1283:1285",7.332,1.298945e-07,SS,0.003313
2,"FM31_955:952,775:781,346:348,1360:1359,1362:1361",7.160,4.513376e-07,SS,0.002004
3,"FM32_1867:1878,219:212,221:226,195:204,211:205...",7.379,1.230310e-08,SS,0.000095
4,FM32_726:735,7.007,2.899940e-06,SS,0.004957
5,"FM32_813:814,802:807,1368:1371,351:349,801:800",7.174,2.794843e-06,SS,0.006632
6,"FM31_-121.1,37.3,0.0_N",6.050,4.963669e-06,N,0.046441
7,"FM31_-120.3,38.4,0.0_N",6.550,1.255875e-06,N,0.044468
8,"FM32_-122.3,37.4,0.0_R",6.150,8.896079e-06,R,0.118782
9,"FM32_-120.7,37.7,0.0_R",7.850,6.058368e-10,R,0.003850


In [ ]:
PGA_selected_events.to_csv(nshmp_hazard_loc+'/PGA_matching_events.csv')

# PGA and Magnitude Matching 

In [ ]:
rDeagg_to_MR_Hazd = robjects.r['Deagg_to_MR_Hazd']
rEvent_to_MR_Hazd = robjects.r['Event_to_MR_Hazd']

In [ ]:
Y_Deagg

In [ ]:
#### Marginal distribution of magnitude based on deaggregation
# Assuming your dataframe is named 'df'
# Group by 'Magnitude' and 'ReturnPeriod', and aggregate sum for 'HazardContribution' and 'Epsilon_bar'
Y_Deagg_drop = Y_Deagg.drop(columns=['R_distance'])

grouped = Y_Deagg_drop.groupby(['ReturnPeriod', 'AnnualExceedanceRate','SiteName','IM_level','Magnitude']).agg({
    'HazardContribution': 'sum',
    'Epsilon_bar': 'sum'
}).reset_index()

pga = ['PGA']*len(grouped)
grouped['IM_type'] = pga
grouped

In [ ]:
with (ro.default_converter + pandas2ri.converter).context():
    Y_Deagg_r = ro.conversion.get_conversion().py2rpy(grouped)

In [ ]:
from rpy2.robjects import r

as_null = r['as.null']
y_M_hazd = rDeagg_to_MR_Hazd(Deagg_df=Y_Deagg_r, R_distance = as_null())

In [ ]:
# y_M_hazd

In [ ]:
y_M_hazd_p = robjects.conversion.rpy2py(y_M_hazd)
y_M_hazd_p

In [ ]:
EventSet.to_csv(nshmp_hazard_loc+'/PGA_eventset.csv')

In [ ]:
# for i in range(1, 158):
#     name = 'Rjb to Site'+str(i)
#     EventSet[name] = EventSet[name].astype(float)
Sitelist = SiteTable[['SiteName', 'Vs30', 'z1']]

values_to_assign = EventSet.iloc[1, 4:].values
Sitelist.loc[:, 'Rjb'] = values_to_assign

Sitelist = Sitelist['Rjb'].astype(float)
Sitelist_r = ro.conversion.py2rpy(Sitelist)

In [ ]:
# values_to_assign = EventSet.iloc[1, 4:].values
# Sitelist.loc[:, 'Rjb'] = values_to_assign
# Sitelist['Rjb'] = Sitelist['Rjb'].astype(float)
# Sitelist_r = ro.conversion.get_conversion().py2rpy(Sitelist)

In [ ]:
# develop X MR hazard distributions from each event in EventSet
# row_num = len(y_M_hazd_p)
# col_num = len(EventSet)
# X_MR = np.zeros([row_num, col_num])

# for i, row in EventSet.iterrows():
#     Fault_type = EventSet['Fault_type']
#     Fault_type_r = ro.conversion.py2rpy(Fault_type)
    
#     Sitelist = SiteTable[['SiteName', 'Vs30', 'z1']]
#     Sitelist['Rjb'] = EventSet.iloc[i, 4:].values
#     values_to_assign = EventSet.iloc[i, 4:].values
#     Sitelist.loc[:, 'Rjb'] = values_to_assign
#     Sitelist['Rjb'] = Sitelist['Rjb'].astype(float)
#     Sitelist_r = ro.conversion.py2rpy(Sitelist)
    
    
    
#     tmp_event_MR = rEvent_to_MR_Hazd(MR_Hazd = y_M_hazd, Mag = row['Magnitude'], 
#                                    AnnualOccurrenceRate = row['AnnualOccurrenceRate'], 
#                                    Fault_type = Fault_type, region = 1, Sitelist = Sitelist_r)
    
#     tmp_event_MR_p = robjects.conversion.rpy2py(y_M_hazd)
#     X_MR[i] = tmp_event_MR_p['Hazd_event'].values

In [ ]:
row = EventSet.loc[0]
Fault_type = EventSet['Fault_type']
Fault_type_r = ro.conversion.get_conversion().py2rpy(Fault_type)

Sitelist = SiteTable[['SiteName','Vs30', 'z1']]
Sitelist_r = ro.conversion.get_conversion().py2rpy(Sitelist)